In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 198 kB 78.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=1996d815de05c1da65e1fa89deba165be91ee7b3b005525a150eb42eebad3526
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
from itertools import chain
from pyspark.sql.functions import count, mean, when, lit, create_map, regexp_extract
from pyspark.sql import Row
from pyspark.sql.session import SparkSession
from pyspark import SparkContext
import pandas as pd
from pyspark.mllib.regression import LabeledPoint
import numpy as np

In [ ]:
sc = SparkContext()
spark = SparkSession(sc)

#**TP simple**

In [ ]:
def clean(file):
    Liste = []
    for ele in file[4:]:
        x = ele.strip('"')
        if x == "?":
            Liste += [float(0)]
        else:
            Liste += [float(x)]
    return Liste

df = sc.textFile("/train_noheader.tsv")

df_split = df.map(lambda x : x.split("\t"))

df_cleaned = df_split.map(lambda x : LabeledPoint(int(clean(x)[-1]),clean(x)[:-1],))

In [ ]:
from pyspark.rdd import RDD
df_cleaned.first()

LabeledPoint(0.0, [0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575])

In [ ]:
df_cleaned.count()

7395

In [ ]:
(training , test) = df_cleaned.randomSplit([0.8,0.2])

In [ ]:
training.cache()
test.cache()

PythonRDD[10] at RDD at PythonRDD.scala:53

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithSGD
model = LogisticRegressionWithSGD.train(training, iterations = 1000 , regParam = 0.05)
LabelPredict = test.map(lambda x : (x.label ,model.predict(x.features) ))
trainingErr = LabelPredict.filter(lambda x : x[0] ==x[1]).count()/float(test.count())
print("accuracy", trainingErr)

/usr/local/lib/python3.7/dist-packages/pyspark/mllib/classification.py:327: FutureWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "LogisticRegressionWithLBFGS.", FutureWarning)


accuracy 0.4943899018232819


In [ ]:
from pyspark.mllib.classification import SVMWithSGD
model2 = SVMWithSGD.train(training, iterations = 1000 , regParam = 0.05)
LabelPredict2 = test.map(lambda x : (x.label ,model2.predict(x.features) ))
trainingErr2 = LabelPredict2.filter(lambda x : x[0] ==x[1]).count()/float(test.count())
print("acc", trainingErr2)

acc 0.4943899018232819


#**Optimisation de la performance**

Je vais opter pour trois approches afin de faire augmenter la performance:


1.   Feature selection : Pearson Correlation
2.   Reduction de la dimension : PCA
3.   Natural Language Processing - NLP ( en prenant compte de la variable boilerplate )



In [ ]:
df2 = sc.textFile("/train_noheader.tsv")

In [ ]:
df2.first()

'"http://www.bloomberg.com/news/2010-12-23/ibm-predicts-holographic-calls-air-breathing-batteries-by-2015.html"\t"4042"\t"{""title"":""IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries"",""body"":""A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose California Photographer Tony Avelar Bloomberg By 2015 your mobile phone will project a 3 D image of anyone who calls and your laptop will be powered by kinetic energy At least that s what International Business Machines Corp sees in its crystal ball The predictions are part of an annual tradition for the Armonk New York based company which surveys its 3 000 researchers to find five ideas expected to take root in the next five years IBM the world s largest 

In [ ]:
parts_it = df2.map(lambda row: row.split("\t"))
print (parts_it.first())

['"http://www.bloomberg.com/news/2010-12-23/ibm-predicts-holographic-calls-air-breathing-batteries-by-2015.html"', '"4042"', '"{""title"":""IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries"",""body"":""A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose California Photographer Tony Avelar Bloomberg By 2015 your mobile phone will project a 3 D image of anyone who calls and your laptop will be powered by kinetic energy At least that s what International Business Machines Corp sees in its crystal ball The predictions are part of an annual tradition for the Armonk New York based company which surveys its 3 000 researchers to find five ideas expected to take root in the next five years IBM the world s lar

In [ ]:
data= parts_it.map(lambda p: Row (url=p[0], urlid=p[1], boilerplate=p[2],  alchemy_category=p[3], alchemy_category_score=(p[4]),
                                 avglinksize=(p[5]), commonlinkratio_1=(p[6]), commonlinkratio_2=(p[7]), commonlinkratio_3=(p[8]), 
                                 commonlinkratio_4=(p[9]), compression_ratio=(p[10]), embed_ratio=(p[11]),
                                 framebased=(p[12]), frameTagRatio=(p[13]), hasDomainLink=(p[14]), html_ratio=p[15], 
                                 image_ratio=(p[16]), is_news=(p[17]),lengthyLinkDomain=(p[18]), linkwordscore=(p[19]), 
                                 news_front_page=(p[20]),non_markup_alphanum_characters=(p[21]), numberOfLinks=(p[22]),
                                 numwords_in_url=(p[23]), parametrizedLinkRatio=(p[24]),
                                  spelling_errors_ratio=(p[25]), label=(p[26])))
print(data.first())


Row(url='"http://www.bloomberg.com/news/2010-12-23/ibm-predicts-holographic-calls-air-breathing-batteries-by-2015.html"', urlid='"4042"', boilerplate='"{""title"":""IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries"",""body"":""A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose California Photographer Tony Avelar Bloomberg By 2015 your mobile phone will project a 3 D image of anyone who calls and your laptop will be powered by kinetic energy At least that s what International Business Machines Corp sees in its crystal ball The predictions are part of an annual tradition for the Armonk New York based company which surveys its 3 000 researchers to find five ideas expected to take root in the next five 

In [ ]:
data=spark.createDataFrame(data)

In [ ]:
data.show()

+--------------------+------+--------------------+--------------------+----------------------+-------------+-----------------+-----------------+-----------------+-----------------+-----------------+-------------+----------+-------------+-------------+-------------+-------------+-------+-----------------+-------------+---------------+------------------------------+-------------+---------------+---------------------+---------------------+-----+
|                 url| urlid|         boilerplate|    alchemy_category|alchemy_category_score|  avglinksize|commonlinkratio_1|commonlinkratio_2|commonlinkratio_3|commonlinkratio_4|compression_ratio|  embed_ratio|framebased|frameTagRatio|hasDomainLink|   html_ratio|  image_ratio|is_news|lengthyLinkDomain|linkwordscore|news_front_page|non_markup_alphanum_characters|numberOfLinks|numwords_in_url|parametrizedLinkRatio|spelling_errors_ratio|label|
+--------------------+------+--------------------+--------------------+----------------------+------------

### **Feature engineering**

In [ ]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import *
import numpy as np
df=data
for colName in data.columns :

  df= df.withColumn(colName,regexp_replace(col(colName), '"', ""))


In [ ]:
df.show()

+--------------------+-----+--------------------+------------------+----------------------+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------+----------+-------------+-------------+-----------+-----------+-------+-----------------+-------------+---------------+------------------------------+-------------+---------------+---------------------+---------------------+-----+
|                 url|urlid|         boilerplate|  alchemy_category|alchemy_category_score|avglinksize|commonlinkratio_1|commonlinkratio_2|commonlinkratio_3|commonlinkratio_4|compression_ratio|embed_ratio|framebased|frameTagRatio|hasDomainLink| html_ratio|image_ratio|is_news|lengthyLinkDomain|linkwordscore|news_front_page|non_markup_alphanum_characters|numberOfLinks|numwords_in_url|parametrizedLinkRatio|spelling_errors_ratio|label|
+--------------------+-----+--------------------+------------------+----------------------+-----------+-----------------+-------

In [ ]:
from pyspark.sql.functions import col,when
for colName in df.columns :
      df=df.withColumn(colName, when(col(colName)=="?" ,None).otherwise(col(colName)))
df.show()

+--------------------+-----+--------------------+------------------+----------------------+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------+----------+-------------+-------------+-----------+-----------+-------+-----------------+-------------+---------------+------------------------------+-------------+---------------+---------------------+---------------------+-----+
|                 url|urlid|         boilerplate|  alchemy_category|alchemy_category_score|avglinksize|commonlinkratio_1|commonlinkratio_2|commonlinkratio_3|commonlinkratio_4|compression_ratio|embed_ratio|framebased|frameTagRatio|hasDomainLink| html_ratio|image_ratio|is_news|lengthyLinkDomain|linkwordscore|news_front_page|non_markup_alphanum_characters|numberOfLinks|numwords_in_url|parametrizedLinkRatio|spelling_errors_ratio|label|
+--------------------+-----+--------------------+------------------+----------------------+-----------+-----------------+-------

In [ ]:
from pyspark.sql.functions import *

def null_value_calc(df):
    null_columns_counts = []
    numRows = df.count()
    for k in df.columns:
        nullRows = df.where(col(k).isNull()).count()
        temp = k,nullRows,(nullRows/numRows)*100
        if nullRows>0:
               null_columns_counts.append(temp)
    return(null_columns_counts)

null_columns_calc_list = null_value_calc(df)
spark.createDataFrame(null_columns_calc_list, ['Column_Name', 'Null_Values_Count','Null_Value_Percent']).show()

+--------------------+-----------------+------------------+
|         Column_Name|Null_Values_Count|Null_Value_Percent|
+--------------------+-----------------+------------------+
|    alchemy_category|             2342|31.670047329276542|
|alchemy_category_...|             2342|31.670047329276542|
|             is_news|             2843|  38.4448951994591|
|     news_front_page|             1248|16.876267748478703|
+--------------------+-----------------+------------------+



In [ ]:
df.select('alchemy_category_score', 'alchemy_category', 'is_news', 'news_front_page').summary('mean', '50%', 'max').show()

+-------+----------------------+----------------+-------+-------------------+
|summary|alchemy_category_score|alchemy_category|is_news|    news_front_page|
+-------+----------------------+----------------+-------+-------------------+
|   mean|    0.6033343166237878|            null|    1.0|0.04782820888238165|
|    50%|              0.625616|            null|    1.0|                0.0|
|    max|              0.999426|         weather|      1|                  1|
+-------+----------------------+----------------+-------+-------------------+



In [ ]:
df = df.fillna({'alchemy_category_score':0.6033343166237878, 'alchemy_category':'weather', 'is_news':0, 'news_front_page':0.04782820888238165})

In [ ]:
df = df.drop('url', 'urlid', 'boilerplate')

In [ ]:
from pyspark.ml.feature import StringIndexer
ac_indexer = StringIndexer(inputCol="alchemy_category", outputCol="alchemi_category")
#Fits a model to the input dataset with optional parameters.
data = ac_indexer.fit(df).transform(df)
data=data.drop('alchemy_category')
data.select('alchemi_category').show()

+----------------+
|alchemi_category|
+----------------+
|             3.0|
|             1.0|
|             4.0|
|             4.0|
|             5.0|
|             0.0|
|             2.0|
|             0.0|
|             0.0|
|             0.0|
|             3.0|
|             5.0|
|             4.0|
|             0.0|
|             1.0|
|             1.0|
|             1.0|
|             2.0|
|             1.0|
|             4.0|
+----------------+
only showing top 20 rows



In [ ]:
data.show(2)

+----------------------+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------+----------+-------------+-------------+-----------+-----------+-------+-----------------+-------------+---------------+------------------------------+-------------+---------------+---------------------+---------------------+-----+----------------+
|alchemy_category_score|avglinksize|commonlinkratio_1|commonlinkratio_2|commonlinkratio_3|commonlinkratio_4|compression_ratio|embed_ratio|framebased|frameTagRatio|hasDomainLink| html_ratio|image_ratio|is_news|lengthyLinkDomain|linkwordscore|news_front_page|non_markup_alphanum_characters|numberOfLinks|numwords_in_url|parametrizedLinkRatio|spelling_errors_ratio|label|alchemi_category|
+----------------------+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------+----------+-------------+-------------+-----------+-----------+-------+-----------------+-

In [ ]:
data=data.drop('framebased')

# **Approche 1: Feature selection 🔔**

In [ ]:
def convert(row):
    Liste = []
    for x in row:
        Liste += [float(x)]
    return Liste

In [ ]:
from pyspark.ml.linalg import Vector as MLVector, Vectors as MLVectors
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors
from pyspark.mllib.regression import  LabeledPoint

labeled_data = data.rdd.map(lambda row : LabeledPoint(int(row[-2]),convert(row)[:-2],))

In [ ]:
labeled_data.first()

LabeledPoint(0.0, [0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575])

### **Pearson correlation coefficients**

Creating first Pearson correlation coefficients (PCCs) between the class and each features (scoreClass), then the PCCs between every pair of feature (scoreMatrix). Once these intermediate results are completed, I proceed into performing the feature selection. 

Here below I define the functions used during the map and reduce stage of the distributed calculations.

In [ ]:
from scipy.stats.stats import pearsonr
from sklearn.metrics import normalized_mutual_info_score
from sklearn.feature_selection import mutual_info_regression
import numpy as np

#This function creates a list of k,v tuples, one per each label-feature combination. 
#'k' corresponds to the index of the feature and 'v' corresponds to a tuple of two elements: value of the label, value of the feature
#lp : labeledPoint
def meltLPclass(lp):
    
    label = lp.label
    features = lp.features
    r = range(features.size)
    return [(i, (label, features[i])) for i in r]


This function calculates the Pearson correlation coefficient among two variables. It returns the index of the feature and its correlation coefficient. the parameter x is tuple 

In [ ]:
def corr(x):
    idx = x[0]
    values = list(x[1])
    
    l = list(values)
    v1, v2 = zip(*values)
    p = pearsonr(v1, v2)[0]
    
    return (idx, p)

This function calculates the Mutual information between two discrete variables. It relies on the 'adjusted_mutual_info_score' function available in the sklearn package.

In [ ]:
def mi(x):
    idx = x[0]
    values = list(x[1])
    
    l = list(values)
    v1, v2 = zip(*values)
    res = normalized_mutual_info_score(v1, v2)
    
    return (idx, res)

This function calculates the Mutual information between two continuous variables. It relies on the 'mutual_info_regression' function available in the sklearn package.

In [ ]:
def miCont(x):
    idx = x[0]
    values = list(x[1])
    
    l = list(values)
    v1, v2 = zip(*values)
    V1 = np.array(v1).reshape(len(v1),1)  # 1-column matrix layout transformation
    res = mutual_info_regression(V1, v2, discrete_features=False, random_state=42)[0]  # random_state is set to provide deterministic results
    
    return (idx, res)

In [ ]:
fscores = labeled_data.flatMap(meltLPclass).groupByKey().map(corr).collect()

In [ ]:
fscores[0:4]

[(0, -0.051758018388009884),
 (2, 0.0833636926283919),
 (4, 0.10596419450867221),
 (6, -0.05973654614063667)]

In [ ]:
ncols = labeled_data.first().features.size  # number of columns (no class) of the dataset
fscoresIdx, fscoresScore = zip(*fscores)
scoreClass = [fscoresScore[fscoresIdx.index(i)] for i in range(ncols)]

In [ ]:
fscoresIdx[0:3]

(0, 2, 4)

In [ ]:
fscoresScore[0:3]

(-0.051758018388009884, 0.0833636926283919, 0.10596419450867221)

In [ ]:
scoreClass[0:3]

[-0.051758018388009884, 0.006172207286742423, 0.0833636926283919]

#### **Top n-features**

In [ ]:
nfs = 10 # number of feature to select

In [ ]:
L=[]
for i in range (len(scoreClass)):
  if scoreClass[i]< 0:
    tup=(scoreClass[i]*(-1),i)
  else:
    tup=(scoreClass[i],i)
  if scoreClass[i]!='nan':
     L.append(tup)
l=sorted(L,key=lambda tup: tup[0], reverse=True)
fsIdx = [x[1] for x in l[0:nfs]]

In [ ]:
fsIdx[0:10]

[8, 14, 4, 16, 3, 2, 5, 17, 6, 20]

In [ ]:
from pyspark.mllib.linalg import Vectors

def reduceLP(lp, fsIdx):
    label = lp.label
    features = lp.features
    v = [features[i] for i in fsIdx]
    return LabeledPoint(label, Vectors.dense(v))

rddFS = labeled_data.map(lambda x: reduceLP(x, fsIdx))

In [ ]:
rddFS.first()

LabeledPoint(0.0, [0.09077381,24.0,0.047058824,5424.0,0.205882353,0.676470588,0.023529412,170.0,0.443783175,0.079129575])

In [ ]:
(training , test) =rddFS.randomSplit([0.8,0.2])

In [ ]:
from pyspark.mllib.classification import SVMWithSGD
model2 = SVMWithSGD.train(training, iterations = 1000 , regParam = 0.005)
LabelPredict2 = test.map(lambda x : (x.label ,model2.predict(x.features) ))
trainingErr2 = LabelPredict2.filter(lambda x : x[0] ==x[1]).count()/float(test.count())
print("acc", trainingErr2)

acc 0.49511173184357543


In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithSGD
model = LogisticRegressionWithSGD.train(training, iterations = 1000 , regParam = 0.05)
LabelPredict = test.map(lambda x : (x.label ,model.predict(x.features) ))
trainingErr = LabelPredict.filter(lambda x : x[0] ==x[1]).count()/float(test.count())
print("accuracy", trainingErr)

/usr/local/lib/python3.7/dist-packages/pyspark/mllib/classification.py:327: FutureWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "LogisticRegressionWithLBFGS.", FutureWarning)


accuracy 0.520682862770847


**Conclusion 1** : Feature selection using pearson correlation coefficients didn't really give us an improvement on the accuracy of both models ! So let's give a try to the second approch

# **Approche 2: PCA 🔔**

In [ ]:
data.show(2)

+----------------------+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------+-------------+-------------+-----------+-----------+-------+-----------------+-------------+---------------+------------------------------+-------------+---------------+---------------------+---------------------+-----+----------------+
|alchemy_category_score|avglinksize|commonlinkratio_1|commonlinkratio_2|commonlinkratio_3|commonlinkratio_4|compression_ratio|embed_ratio|frameTagRatio|hasDomainLink| html_ratio|image_ratio|is_news|lengthyLinkDomain|linkwordscore|news_front_page|non_markup_alphanum_characters|numberOfLinks|numwords_in_url|parametrizedLinkRatio|spelling_errors_ratio|label|alchemi_category|
+----------------------+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------+-------------+-------------+-----------+-----------+-------+-----------------+-------------+---------------+----

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType
for i in data.columns:
  data=data.withColumn(i, col(i).cast(FloatType()))

In [ ]:
data.printSchema()

root
 |-- alchemy_category_score: float (nullable = true)
 |-- avglinksize: float (nullable = true)
 |-- commonlinkratio_1: float (nullable = true)
 |-- commonlinkratio_2: float (nullable = true)
 |-- commonlinkratio_3: float (nullable = true)
 |-- commonlinkratio_4: float (nullable = true)
 |-- compression_ratio: float (nullable = true)
 |-- embed_ratio: float (nullable = true)
 |-- frameTagRatio: float (nullable = true)
 |-- hasDomainLink: float (nullable = true)
 |-- html_ratio: float (nullable = true)
 |-- image_ratio: float (nullable = true)
 |-- is_news: float (nullable = true)
 |-- lengthyLinkDomain: float (nullable = true)
 |-- linkwordscore: float (nullable = true)
 |-- news_front_page: float (nullable = true)
 |-- non_markup_alphanum_characters: float (nullable = true)
 |-- numberOfLinks: float (nullable = true)
 |-- numwords_in_url: float (nullable = true)
 |-- parametrizedLinkRatio: float (nullable = true)
 |-- spelling_errors_ratio: float (nullable = true)
 |-- label: floa

In [ ]:
(training , test) =data.randomSplit([0.8,0.2])

In [ ]:
# Cast to vectors so that PCA can work with our data
from pyspark.ml.feature import VectorAssembler

ignore = ['label']

assembler = VectorAssembler(
    inputCols=[x for x in training.columns if x not in ignore],
    outputCol='features')

train_vec = assembler.transform(training).select(['label', 'features'])
test_vec = assembler.transform(test).select(['label', 'features'])

In [ ]:
from pyspark.ml.feature import MinMaxScaler
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

# Compute summary statistics and generate MinMaxScalerModel
scalerModel = scaler.fit(train_vec)

# rescale each feature to range [min, max].
Trainscaled_data = scalerModel.transform(train_vec)
# Compute summary statistics and generate MinMaxScalerModel
#scalerModel = scaler.fit(test_vec)

# rescale each feature to range [min, max].
testscaled_data = scalerModel.transform(test_vec)

In [ ]:
# Perform PCA
from pyspark.ml.feature import PCA

pca = PCA(k=10, inputCol="scaledFeatures", outputCol = "pca_features")

mdl_pca = pca.fit(Trainscaled_data)
train_pca = mdl_pca.transform(Trainscaled_data)
test_pca = mdl_pca.transform(testscaled_data)

mdl_pca.explainedVariance

DenseVector([0.3015, 0.2209, 0.1304, 0.0576, 0.0521, 0.0459, 0.0395, 0.0359, 0.0328, 0.0225])

In [ ]:
train_pca.show(2)

+-----+--------------------+--------------------+--------------------+
|label|            features|      scaledFeatures|        pca_features|
+-----+--------------------+--------------------+--------------------+
|  1.0|[0.07840909808874...|[0.00815836159370...|[-0.0650823735728...|
|  1.0|[0.07840909808874...|[0.00815836159370...|[-0.8158975305454...|
+-----+--------------------+--------------------+--------------------+
only showing top 2 rows



In [ ]:
train_pca=train_pca.drop('features')
test_pca=test_pca.drop('features')
train_pca=train_pca.drop('scaledFeatures')
test_pca=test_pca.drop('scaledFeatures')

In [ ]:
train_pca.show(2)

+-----+--------------------+
|label|        pca_features|
+-----+--------------------+
|  1.0|[-0.0650823735728...|
|  1.0|[-0.8158975305454...|
+-----+--------------------+
only showing top 2 rows



In [ ]:
training = train_pca.rdd.map(lambda row : LabeledPoint(int(row[0]),MLLibVectors.fromML(row[1]),))

In [ ]:
test=test_pca.rdd.map(lambda row : LabeledPoint(int(row[0]),MLLibVectors.fromML(row[1]),))

In [ ]:
training.first()

LabeledPoint(1.0, [-0.06508237357285881,0.09013824837629196,-0.1579084128297016,0.35099565683783807,0.18773342000336046,-0.10465300211225166,-0.0001622720656552501,-0.10464364920685171,0.008907928337961432,-0.010285004811913655])

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithSGD
model = LogisticRegressionWithSGD.train(training, iterations = 1000 , regParam = 0.05, regType='l2', miniBatchFraction=0.01)
LabelPredict = test.map(lambda x : (x.label ,model.predict(x.features) ))
trainingErr = LabelPredict.filter(lambda x : x[0] ==x[1]).count()/float(test.count())
print("accuracy", trainingErr)

/usr/local/lib/python3.7/dist-packages/pyspark/mllib/classification.py:327: FutureWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "LogisticRegressionWithLBFGS.", FutureWarning)


accuracy 0.6205766710353866


In [ ]:
from pyspark.mllib.classification import SVMWithSGD
model2 = SVMWithSGD.train(training, iterations = 1000 , regParam = 0.05, regType='l2', miniBatchFraction=0.01)
LabelPredict2 = test.map(lambda x : (x.label ,model2.predict(x.features) ))
trainingErr2 = LabelPredict2.filter(lambda x : x[0] ==x[1]).count()/float(test.count())
print("acc", trainingErr2)

acc 0.5563564875491481


##### **What could be the best value of regParam that could give us the maximum accuracy in this case ?**

In [ ]:
def modelWithSVM(trainingData, validationData):
  reg=[ 0.001, 0.005,0.01, 0.002, 0.0009]
  bestSVMModel = None
  bestAccuracy = 0
  numOfIterations = 1000
  visualizationData = []
  for regularizer in reg:
    model = SVMWithSGD.train(trainingData, numOfIterations, regParam=regularizer, miniBatchFraction=0.01)
    predict = validationData.map(lambda ad: (ad.label, model.predict(ad.features)))
    totalValidationAds = validationData.count()
    correctlyPredicted = predict.filter(lambda x: x[0] == x[1]).count()
    accuracy = float(correctlyPredicted)/totalValidationAds
    visualizationData += [(regularizer, accuracy)]
    if accuracy > bestAccuracy:
      bestAccuracy = accuracy
      bestSVMModel = model
  return(visualizationData)

In [ ]:
print(modelWithSVM(training, test))

[(0.001, 0.6035386631716907), (0.005, 0.6055045871559633), (0.01, 0.59043250327654), (0.002, 0.6120576671035387), (0.0009, 0.6035386631716907)]


on peut remarquer que regparam=0.002 donne la meilleure performance. La performance a augmenté de 55% à 61%

In [ ]:
from pyspark.mllib.classification import SVMWithSGD
model2 = SVMWithSGD.train(training, iterations = 1000 , regParam = 0.002, regType='l2', miniBatchFraction=0.01)
LabelPredict2 = test.map(lambda x : (x.label ,model2.predict(x.features) ))
trainingErr2 = LabelPredict2.filter(lambda x : x[0] ==x[1]).count()/float(test.count())
print("acc", trainingErr2)

acc 0.6120576671035387


In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithSGD
model = LogisticRegressionWithSGD.train(training, iterations = 1000 , regParam = 0.002, regType='l2', miniBatchFraction=0.01)
LabelPredict = test.map(lambda x : (x.label ,model.predict(x.features) ))
trainingErr = LabelPredict.filter(lambda x : x[0] ==x[1]).count()/float(test.count())
print("accuracy", trainingErr)

/usr/local/lib/python3.7/dist-packages/pyspark/mllib/classification.py:327: FutureWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "LogisticRegressionWithLBFGS.", FutureWarning)


accuracy 0.6297509829619922


**Conclusion 2**: PCA has improved the accuracy of both models : 
-> 49% to 62% 🙂

# **Approche 3 : NLP 🔔**

In [ ]:
df2 = sc.textFile("/train_noheader.tsv")

In [ ]:
df2.first()

'"http://www.bloomberg.com/news/2010-12-23/ibm-predicts-holographic-calls-air-breathing-batteries-by-2015.html"\t"4042"\t"{""title"":""IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries"",""body"":""A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose California Photographer Tony Avelar Bloomberg By 2015 your mobile phone will project a 3 D image of anyone who calls and your laptop will be powered by kinetic energy At least that s what International Business Machines Corp sees in its crystal ball The predictions are part of an annual tradition for the Armonk New York based company which surveys its 3 000 researchers to find five ideas expected to take root in the next five years IBM the world s largest 

In [ ]:
parts_it = df2.map(lambda row: row.split("\t"))
print (parts_it.first())

['"http://www.bloomberg.com/news/2010-12-23/ibm-predicts-holographic-calls-air-breathing-batteries-by-2015.html"', '"4042"', '"{""title"":""IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries"",""body"":""A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose California Photographer Tony Avelar Bloomberg By 2015 your mobile phone will project a 3 D image of anyone who calls and your laptop will be powered by kinetic energy At least that s what International Business Machines Corp sees in its crystal ball The predictions are part of an annual tradition for the Armonk New York based company which surveys its 3 000 researchers to find five ideas expected to take root in the next five years IBM the world s lar

In [ ]:
data= parts_it.map(lambda p: Row (url=p[0], urlid=p[1], boilerplate=p[2],  alchemy_category=p[3]))
print(data.first())


Row(url='"http://www.bloomberg.com/news/2010-12-23/ibm-predicts-holographic-calls-air-breathing-batteries-by-2015.html"', urlid='"4042"', boilerplate='"{""title"":""IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries"",""body"":""A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose California Photographer Tony Avelar Bloomberg By 2015 your mobile phone will project a 3 D image of anyone who calls and your laptop will be powered by kinetic energy At least that s what International Business Machines Corp sees in its crystal ball The predictions are part of an annual tradition for the Armonk New York based company which surveys its 3 000 researchers to find five ideas expected to take root in the next five 

In [ ]:
data_nlp=spark.createDataFrame(data)

In [ ]:
data_nlp.show()

+--------------------+------+--------------------+--------------------+
|                 url| urlid|         boilerplate|    alchemy_category|
+--------------------+------+--------------------+--------------------+
|"http://www.bloom...|"4042"|"{""title"":""IBM...|          "business"|
|"http://www.popsc...|"8471"|"{""title"":""The...|        "recreation"|
|"http://www.mensh...|"1164"|"{""title"":""Fru...|            "health"|
|"http://www.dumbl...|"6684"|"{""title"":""10 ...|            "health"|
|"http://bleacherr...|"9006"|"{""title"":""The...|            "sports"|
|"http://www.conve...|"7018"|"{""url"":""conve...|                 "?"|
|"http://gofashion...|"8685"|"{""title"":""fas...|"arts_entertainment"|
|"http://www.insid...|"3402"|"{""url"":""insid...|                 "?"|
|"http://www.valet...| "477"|"{""title"":""Val...|                 "?"|
|"http://www.howsw...|"6731"|"{""url"":""howsw...|                 "?"|
|"http://www.reute...|"2293"|"{""title"":""Bus...|          "bus

**Cleaning data_nlp**

In [ ]:
from pyspark.sql.functions import *

def null_value_calc(df):
    null_columns_counts = []
    numRows = df.count()
    for k in df.columns:
        nullRows = df.where(col(k).isNull()).count()
        temp = k,nullRows,(nullRows/numRows)*100
        null_columns_counts.append(temp)
    return(null_columns_counts)

null_columns_calc_list = null_value_calc(data_nlp)
spark.createDataFrame(null_columns_calc_list, ['Column_Name', 'Null_Values_Count','Null_Value_Percent']).show()

+----------------+-----------------+------------------+
|     Column_Name|Null_Values_Count|Null_Value_Percent|
+----------------+-----------------+------------------+
|             url|                0|               0.0|
|           urlid|                0|               0.0|
|     boilerplate|                0|               0.0|
|alchemy_category|                0|               0.0|
+----------------+-----------------+------------------+



In [ ]:
from pyspark.ml.feature import * #CountVectorizer,StringIndexer, RegexTokenizer,StopWordsRemover
from pyspark.sql.functions import * #col, udf,regexp_replace,isnull
from pyspark.sql.types import * #StringType,IntegerType
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# For pipeline development
from pyspark.ml import Pipeline 

In [ ]:
# Remove ""
df = data_nlp.withColumn("alchemy_category",regexp_replace(col('alchemy_category'), '"', ""))
df1 = df.withColumn("url",regexp_replace(col('url'), '"', ""))
df3 = df1.withColumn("urlid",regexp_replace(col('urlid'), '"', ""))
df4 = df3.withColumn("boilerplate",regexp_replace(col('boilerplate'), '"', ""))



df4.show(8,False)

+----------------------------------------------------------------------------------------------------------------------+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
data_nlp=df4

**we got urlid so I think it's better to delete the column url since we have a column that refers to it**

In [ ]:
data_nlp=data_nlp.drop('url')
data_nlp.show()

+-----+--------------------+------------------+
|urlid|         boilerplate|  alchemy_category|
+-----+--------------------+------------------+
| 4042|{title:IBM Sees H...|          business|
| 8471|{title:The Fully ...|        recreation|
| 1164|{title:Fruits tha...|            health|
| 6684|{title:10 Foolpro...|            health|
| 9006|{title:The 50 Coo...|            sports|
| 7018|{url:conveniencem...|                 ?|
| 8685|{title:fashion la...|arts_entertainment|
| 3402|{url:insidersheal...|                 ?|
|  477|{title:Valet The ...|                 ?|
| 6731|{url:howsweeteats...|                 ?|
| 2293|{title:Business F...|          business|
| 3651|{title:A Tip of t...|            sports|
| 1917|{title:9 Foods Th...|            health|
| 1063|{title: ,body: ,u...|                 ?|
| 2110|{url:phillyburbs ...|        recreation|
| 5318|{title:Izabel Gou...|        recreation|
| 9842|{title:Liquid Mou...|        recreation|
| 3946|{title:null,body:...|arts_enterta

 **alchemy_category is clearly a categorical column so let's apply one-hot encoding to it! In this case, I think there is no need to replace '?' with a specific value since the column is gonna be replaced with a new column.** 

In [ ]:
from pyspark.ml.feature import StringIndexer
ac_indexer = StringIndexer(inputCol="alchemy_category", outputCol="ac_Index")
#Fits a model to the input dataset with optional parameters.
data_nlp = ac_indexer.fit(data_nlp).transform(data_nlp)
data_nlp.show()

+-----+--------------------+------------------+--------+
|urlid|         boilerplate|  alchemy_category|ac_Index|
+-----+--------------------+------------------+--------+
| 4042|{title:IBM Sees H...|          business|     3.0|
| 8471|{title:The Fully ...|        recreation|     1.0|
| 1164|{title:Fruits tha...|            health|     4.0|
| 6684|{title:10 Foolpro...|            health|     4.0|
| 9006|{title:The 50 Coo...|            sports|     5.0|
| 7018|{url:conveniencem...|                 ?|     0.0|
| 8685|{title:fashion la...|arts_entertainment|     2.0|
| 3402|{url:insidersheal...|                 ?|     0.0|
|  477|{title:Valet The ...|                 ?|     0.0|
| 6731|{url:howsweeteats...|                 ?|     0.0|
| 2293|{title:Business F...|          business|     3.0|
| 3651|{title:A Tip of t...|            sports|     5.0|
| 1917|{title:9 Foods Th...|            health|     4.0|
| 1063|{title: ,body: ,u...|                 ?|     0.0|
| 2110|{url:phillyburbs ...|   

Now, we can delete alchemy_category

In [ ]:
data_nlp=data_nlp.drop('alchemy_category')
data_nlp.show()

+-----+--------------------+--------+
|urlid|         boilerplate|ac_Index|
+-----+--------------------+--------+
| 4042|{title:IBM Sees H...|     3.0|
| 8471|{title:The Fully ...|     1.0|
| 1164|{title:Fruits tha...|     4.0|
| 6684|{title:10 Foolpro...|     4.0|
| 9006|{title:The 50 Coo...|     5.0|
| 7018|{url:conveniencem...|     0.0|
| 8685|{title:fashion la...|     2.0|
| 3402|{url:insidersheal...|     0.0|
|  477|{title:Valet The ...|     0.0|
| 6731|{url:howsweeteats...|     0.0|
| 2293|{title:Business F...|     3.0|
| 3651|{title:A Tip of t...|     5.0|
| 1917|{title:9 Foods Th...|     4.0|
| 1063|{title: ,body: ,u...|     0.0|
| 2110|{url:phillyburbs ...|     1.0|
| 5318|{title:Izabel Gou...|     1.0|
| 9842|{title:Liquid Mou...|     1.0|
| 3946|{title:null,body:...|     2.0|
|  334|{title:Grilled Pe...|     1.0|
|  271|{title:How to Mak...|     4.0|
+-----+--------------------+--------+
only showing top 20 rows



**Clean the boilerplate column**

In [ ]:
# Replace Slashes and parenthesis with spaces
# You can test your script on line 7 of the df "(Legend of Zelda/Fable Inspired)"
data_nlp = data_nlp.withColumn("boilerplate",translate(col("boilerplate"), ":", " ")) \
        .withColumn("boilerplate",translate(col("boilerplate"), "{", " ")) \
        .withColumn("boilerplate",translate(col("boilerplate"), "}", " "))
data_nlp.select("boilerplate").show(7,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Removing anything that is not a letter
df = data_nlp.withColumn("boilerplate",regexp_replace(col('boilerplate'), '[^A-Za-z ]+', ''))
df.select("boilerplate").show(10,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
df = df.withColumn("boilerplate",regexp_replace(col('boilerplate'), 'title', ''))
df = df.withColumn("boilerplate",regexp_replace(col('boilerplate'), 'url', ''))
data_nlp = df.withColumn("boilerplate",regexp_replace(col('boilerplate'), 'body', ''))
data_nlp.select("boilerplate").show(10,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Remove multiple spaces
data_nlp = data_nlp.withColumn("boilerplate",regexp_replace(col('boilerplate'), ' +', ' '))
data_nlp.select("boilerplate").show(4,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Lower case everything
data_nlp= data_nlp.withColumn("boilerplate",lower(col('boilerplate')))
data_nlp.select("boilerplate").show(4,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# **NLP** 🦄

Split text into words (Tokenizing)

In [ ]:
regex_tokenizer = RegexTokenizer(inputCol="boilerplate", outputCol="words", pattern="\\W")
raw_words = regex_tokenizer.transform(data_nlp)
raw_words.show(2,False)
raw_words.printSchema()

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Removing Stopwords

In [ ]:
# from pyspark.ml.feature import StopWordsRemover

# Define a list of stop words or use default list
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
stopwords = remover.getStopWords() 
  
# Display default list
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']

In [ ]:
words_df = remover.transform(raw_words)
words_df.show(1)

+-----+--------------------+--------+--------------------+--------------------+
|urlid|         boilerplate|ac_Index|               words|            filtered|
+-----+--------------------+--------+--------------------+--------------------+
| 4042| ibm sees hologra...|     3.0|[ibm, sees, holog...|[ibm, sees, holog...|
+-----+--------------------+--------+--------------------+--------------------+
only showing top 1 row



In [ ]:
# Create the Pipeline
pipeline = Pipeline(stages=[regex_tokenizer,remover])
data_prep_pl = pipeline.fit(data_nlp)

# Now call on the Pipeline to get our final df
feature_data = data_prep_pl.transform(data_nlp)
feature_data.show(1)

+-----+--------------------+--------+--------------------+--------------------+
|urlid|         boilerplate|ac_Index|               words|            filtered|
+-----+--------------------+--------+--------------------+--------------------+
| 4042| ibm sees hologra...|     3.0|[ibm, sees, holog...|[ibm, sees, holog...|
+-----+--------------------+--------+--------------------+--------------------+
only showing top 1 row



**Converting text into vectors**

I will test out the **Word2Vec** vectorizor

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number
from pyspark.sql import functions as F

In [ ]:
data_tp5=spark.createDataFrame(df_cleaned)

In [ ]:
df = data_tp5.withColumn('row_id',F.monotonically_increasing_id())
df.show()

+--------------------+-----+------+
|            features|label|row_id|
+--------------------+-----+------+
|[0.789131,2.05555...|  0.0|     0|
|[0.574147,3.67796...|  1.0|     1|
|[0.996526,2.38288...|  1.0|     2|
|[0.801248,1.54310...|  1.0|     3|
|[0.719157,2.67647...|  0.0|     4|
|[0.0,119.0,0.7454...|  0.0|     5|
|[0.22111,0.773809...|  1.0|     6|
|[0.0,1.883333333,...|  0.0|     7|
|[0.0,0.471502591,...|  1.0|     8|
|[0.0,2.41011236,0...|  1.0|     9|
|[0.816604,2.50652...|  0.0|    10|
|[0.89156,4.986111...|  0.0|    11|
|[0.872323,3.05691...|  1.0|    12|
|[0.0,0.0,0.0,0.0,...|  1.0|    13|
|[0.559991,2.29949...|  0.0|    14|
|[0.548963,0.99043...|  0.0|    15|
|[0.598149,0.92982...|  0.0|    16|
|[0.77292,2.083333...|  0.0|    17|
|[0.287884,3.16438...|  1.0|    18|
|[0.585389,2.82677...|  1.0|    19|
+--------------------+-----+------+
only showing top 20 rows



In [ ]:
# Word2Vec
word2Vec = Word2Vec(vectorSize=15, minCount=0, inputCol="filtered", outputCol="features")
model = word2Vec.fit(feature_data)

W2VfeaturizedData = model.transform(feature_data)

scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

# Compute summary statistics and generate MinMaxScalerModel
scalerModel = scaler.fit(W2VfeaturizedData)

# rescale each feature to range [min, max].
scaled_data = scalerModel.transform(W2VfeaturizedData)
W2VfeaturizedData = scaled_data.select('urlid','ac_index','scaledFeatures')
W2VfeaturizedData = W2VfeaturizedData.withColumnRenamed('scaledFeatures','features1')

#joining both dataframes in order to apply classification models

#adding row_id to data_nlp so that I can easily join it with dataframe containing features (5->22) and target
df1 = W2VfeaturizedData.withColumn('row_id',F.monotonically_increasing_id())
W2VfeaturizedData = df1.join(df, ['row_id'], 'left')
#deleting row_id, ac_index and urlid after joining
W2VfeaturizedData = W2VfeaturizedData.drop('row_id')
#W2VfeaturizedData = W2VfeaturizedData.drop('ac_index')
W2VfeaturizedData = W2VfeaturizedData.drop('urlid')
W2VfeaturizedData = W2VfeaturizedData.drop('features')
#W2VfeaturizedData =W2VfeaturizedData.withColumn("features", concat(col("features1"),col("features")))
W2VfeaturizedData = W2VfeaturizedData.withColumnRenamed('features1','features')

W2VfeaturizedData.name = 'W2VfeaturizedData' # We will need this to print later

In [ ]:
W2VfeaturizedData.show(2)

+--------+--------------------+-----+
|ac_index|            features|label|
+--------+--------------------+-----+
|     3.0|[0.67583097061772...|  0.0|
|     1.0|[0.58394053032566...|  1.0|
+--------+--------------------+-----+
only showing top 2 rows



In [ ]:
W2VfeaturizedData.select('features').show()

+--------------------+
|            features|
+--------------------+
|[0.65520995706264...|
|[0.67583097061772...|
|[0.56898178711883...|
|[0.55267512339896...|
|[0.52639653439018...|
|[0.65943147184192...|
|[0.68070165355285...|
|[0.58394053032566...|
|[0.65833521889253...|
|[0.52076031896703...|
|[0.55501059298462...|
|[0.54475502905676...|
|[0.53757659035358...|
|[0.70983484955931...|
|[0.57145139841587...|
|[0.41304890792798...|
|[0.53326572117392...|
|[0.61406021407483...|
|[0.55379945010500...|
|[0.61677608500293...|
+--------------------+
only showing top 20 rows



**Train and evaluate our models**

In [ ]:
W2VfeaturizedData.show(1)

+--------+--------------------+-----+
|ac_index|            features|label|
+--------+--------------------+-----+
|     3.0|[0.67583097061772...|  0.0|
+--------+--------------------+-----+
only showing top 1 row



In [ ]:
from pyspark.ml.linalg import Vector as MLVector, Vectors as MLVectors
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors
from pyspark.mllib.regression import  LabeledPoint
#labeled = W2VfeaturizedData.rdd.map(lambda row : LabeledPoint(row.get("label"), row.get("features")))
labeled = W2VfeaturizedData.rdd.map(lambda row : LabeledPoint(int(row[2]),MLLibVectors.fromML(row[1]),))

In [ ]:
(training , test) =labeled.randomSplit([0.8,0.2])

In [ ]:

labeled.first()

LabeledPoint(0.0, [0.6158325789764636,0.5929008474103011,0.6044689458569324,0.350053691883606,0.6458054635104243,0.5996900292465736,0.6390490194832239,0.3058508499687777,0.6522262589138477,0.6844004773424611,0.505477303125482,0.580975805911345,0.5101277942451384,0.4874318628078161,0.364511244369038])

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithSGD
model = LogisticRegressionWithSGD.train(training, iterations = 1000 , regParam = 0.05)
LabelPredict = test.map(lambda x : (x.label ,model.predict(x.features) ))
trainingErr = LabelPredict.filter(lambda x : x[0] ==x[1]).count()/float(test.count())
print("accuracy", trainingErr)

/usr/local/lib/python3.7/dist-packages/pyspark/mllib/classification.py:327: FutureWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "LogisticRegressionWithLBFGS.", FutureWarning)


accuracy 0.8066666666666666


In [ ]:
from pyspark.mllib.classification import SVMWithSGD
model2 = SVMWithSGD.train(training, iterations = 1000 , regParam = 0.05)
LabelPredict2 = test.map(lambda x : (x.label ,model2.predict(x.features) ))
trainingErr2 = LabelPredict2.filter(lambda x : x[0] ==x[1]).count()/float(test.count())
print("acc", trainingErr2)

acc 0.71


###**essayons de regler les parametres des 2 modèles**

model 1 : LogisticRegressionWithSGD

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithSGD
model = LogisticRegressionWithSGD.train(training, iterations = 1000 , regParam = 0.005)
LabelPredict = test.map(lambda x : (x.label ,model.predict(x.features) ))
trainingErr = LabelPredict.filter(lambda x : x[0] ==x[1]).count()/float(test.count())
print("accuracy", trainingErr)

/usr/local/lib/python3.7/dist-packages/pyspark/mllib/classification.py:327: FutureWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "LogisticRegressionWithLBFGS.", FutureWarning)


accuracy 0.81


model 2: SVMWithSGD 

In [ ]:
from pyspark.mllib.classification import SVMWithSGD
model2 = SVMWithSGD.train(training, iterations = 1000 , regParam = 0.005, miniBatchFraction=0.01)
LabelPredict2 = test.map(lambda x : (x.label ,model2.predict(x.features) ))
trainingErr2 = LabelPredict2.filter(lambda x : x[0] ==x[1]).count()/float(test.count())
print("acc", trainingErr2)

acc 0.8


avec un regParam=0.005 et miniBatchFraction=0.01, la performance du modèle est devenue 80%! Which is good 👏

**Conclusion 3** : After applying NLP , the accuracy has really improved for both models : it used to be 49% but it has became 81% 🙂

### **General conclusion**:  NLP wins 🥇 with 81% , PCA got the second place 🥈 with 61%, then there is Pearson correlation which accuracy has not been improved !